# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [2]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
for key, value in obj.items():
    print(f"{key:10} = '{value}'")


home_page  = 'https://github.com/pypa/sampleproject'
keywords   = 'sample setuptools development'
license    = 'MIT'


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [5]:
import pandas as pd
import re

obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])

pattern = r"\d{2}-\d"

for i in obj:
    match = re.search(pattern, i)
    if match:
        print(match.group())

19-1
20-4
20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [6]:
text = "Написать регулярное выражение, которое позволит найти номера групп студентов."
words = text.split()
print(words)

['Написать', 'регулярное', 'выражение,', 'которое', 'позволит', 'найти', 'номера', 'групп', 'студентов.']


## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [12]:
import pandas as pd
from prettytable import PrettyTable

recipes = pd.read_csv("recipes_sample.csv")

sample_recipes = recipes.sample(n=5)

table = PrettyTable()
table.field_names = ["id", "minutes"]

for _, row in sample_recipes.iterrows():
    table.add_row([row['id'], row['minutes']])

print(table)


+--------+---------+
|   id   | minutes |
+--------+---------+
| 40577  |    10   |
| 246269 |    55   |
| 36537  |    80   |
| 310170 |   122   |
|  383   |    20   |
+--------+---------+


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [17]:
import pandas as pd
import xml.dom.minidom as minidom

df_recipes = pd.read_csv('recipes_sample.csv')

dom = minidom.parse('steps_sample.xml')
root = dom.documentElement

steps_dict = {}
recipes = root.getElementsByTagName('recipe')
for recipe in recipes:
    recipe_id = int(recipe.getElementsByTagName('id')[0].firstChild.data)
    steps = [step.firstChild.data for step in recipe.getElementsByTagName('step')]
    steps_dict[recipe_id] = steps

def show_info(recipe_id):
    row = df_recipes[df_recipes['id'] == recipe_id]
    name = row['name'].iloc[0]
    minutes = row['minutes'].iloc[0]
    author_id = row['contributor_id'].iloc[0]
    steps = steps_dict[recipe_id]

    tot= f'"{name.title()}"\n\n'
    for index, step in enumerate(steps):
        tot += f'{index+1}. {step.capitalize()}\n'
    tot += '-' * 10 + '\n'
    tot += f'Автор: {author_id}\n'
    tot += f'Среднее время приготовления: {minutes} минут\n'

    return tot

print(show_info(170895))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [19]:
import re
import xml.etree.ElementTree as ET

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

recipe_id = '25082'

for recipe in root.findall(".//recipe[id='" + recipe_id + "']"):
    steps = recipe.findall("steps/step")
    for i, step in enumerate(steps, 1):
        step_text = step.text
        matches = re.findall(r'\d+\s+(?:hour|minute)s?', step_text)
        if matches:
            time_info = ', '.join(matches)
            print(f"Шаг {i}: {time_info}")

Шаг 6: 20 minutes
Шаг 8: 10 minutes
Шаг 10: 2 hours
Шаг 14: 10 minutes
Шаг 17: 20 minutes, 30 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [28]:
import pandas as pd

df = pd.read_csv('recipes_sample.csv')
df['description'].fillna(' ', inplace=True)

pt = '^this[\w\d\s]+,[ ]?but'
matches = df[df['description'].str.match(pt, na=False)]

print('Количество подходящих описаний:', matches.shape[0])
pd.set_option('max_colwidth', int(matches['description'].apply(len).max() + 10))
print(matches['description'].sample(3))

Количество подходящих описаний: 134
4151     this cake is very rich, but delicious.  it is great for family gatherings (my family always begs me to make one).  i got the recipe from an older lady i worked with.
2646                                                                                                 this is considered peasant soup, but i consider it queen in comfort food.
21471                                                     this is a different, but super combo, the taste is fantastic, i am quite sure that you will love this recipe. enjoy!
Name: description, dtype: object


In [30]:
import pandas as pd

df = pd.read_csv('recipes_sample.csv')
df['description'].fillna(' ', inplace=True)

ptrn = '^this[\w\d\s]+,[ ]?but'
matches = df[df['description'].str.match(ptrn, na=False)]

print('Количество подходящих описаний:', matches.shape[0])
pd.set_option('max_colwidth', int(matches['description'].apply(len).max() + 10))
print(matches['description'].sample(3))

Количество подходящих описаний: 134
28159                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           this recipe is a little bit of work, but i think it's worth it for good food.  serve with cooked rice and chopped scallions for garnish.\r\n
28090                                                                                                                                                                                                                                                                                                                                  

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [31]:
import pandas as pd
import xml.etree.ElementTree as ET
import re

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

recipe_id = '72367'
recipe_steps = root.findall(".//recipe[id='" + recipe_id + "']/steps/step")

for step in recipe_steps:
    step_text = step.text
    if step_text:
        step_text = re.sub(r'\s*/\s*', '/', step_text)
        step.text = step_text

for step in recipe_steps:
    print(step.text)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [34]:
import re

recipe_steps = {}
for recipe in ab.find_all('recipe'):
    recipe_id = int(recipe.id.text)
    steps = [step.text for step in recipe.find_all('step')]
    recipe_steps[recipe_id] = steps

# Объединение шагов в одну строку и приведение к нижнему регистру
flat_steps = ' '.join(step.lower() for steps in recipe_steps.values() for step in steps)

# Извлечение слов с использованием регулярного выражения
word_list = re.findall(r'\b\w+\b', flat_steps)

# Создание множества уникальных слов
unique_word_set = set(word_list)

print(f"Количество уникальных слов: {len(unique_word_set)}")

Количество уникальных слов: 16109


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
import pandas as pd
import nltk

recipes = pd.read_csv('recipes_sample.csv')

recipes['description'] = recipes['description'].fillna('')

# Функция для подсчета количества предложений
def count_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return len(sentences)

# Применяем функцию для подсчета количества предложений к столбцу 'description'
recipes['num_sentences'] = recipes['description'].apply(count_sentences)

# Сортируем DataFrame по количеству предложений в порядке убывания
recipes_sorted = recipes.sort_values(by='num_sentences', ascending=False)

# Выводим первые 5 описаний с наибольшим количеством предложений
for desc in recipes_sorted.head(5)['description']:
    print(desc)
    print('-' * 80)

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
import nltk

def print_pos_tags(sentence):
    tokens = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(tokens)
    for word, pos in pos_tags:
        print("{0:^10} {1}".format(pos, word))

title = df.loc[df['id'] == 241106, 'name'].iloc[0]

print_pos_tags(title)